# Simple Reasoning Chatbot Demo

This notebook demonstrates the v1 reasoning chatbot that shows its thinking process before answering.

In [ ]:
# Setup and imports
import sys
import os
from pathlib import Path
import uuid
from typing import List, Tuple
from IPython.display import display, Markdown

# Add parent directory to path
notebook_dir = Path.cwd()
parent_dir = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
sys.path.insert(0, str(parent_dir))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import modules
from src.graph import create_graph
from src.state import State
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
# Create the graph
workflow = create_graph()

In [ ]:
# Helper class for managing sessions
class ReasoningChatbot:
    def __init__(self):
        self.workflow = create_graph()
        self.session_id = str(uuid.uuid4())
        self.history: List[Tuple[str, str]] = []
        
    def ask(self, question: str) -> str:
        """Ask a question and get a reasoned response"""
        
        # Create thread ID
        thread_id = str(uuid.uuid4())
        
        # Initialize state
        initial_state: State = {
            "session_id": self.session_id,
            "thread_id": thread_id,
            "messages": [],
            "history": self.history,
            "current_question": question,
            "reasoning_steps": [],
            "reasoning_count": 0,
            "ready_to_answer": False,
            "context": {},
            "tools": [],
            "final_answer": None
        }
        
        # Run the workflow with config
        config = {"configurable": {"thread_id": thread_id}}
        
        display(Markdown(f"## Question\n{question}"))
        display(Markdown("---"))
        
        # Run the workflow
        final_state = self.workflow.invoke(initial_state, config)
        
        # Display reasoning steps (always at least 2)
        if final_state.get("reasoning_steps"):
            display(Markdown("### Reasoning Process"))
            for i, step in enumerate(final_state["reasoning_steps"], 1):
                display(Markdown(f'<span style="color: #FFC107;">**Step {i}:** {step.content}</span>'))
        
        # Get final answer
        final_answer = final_state.get("final_answer", "No answer generated")
        
        # Update history
        self.history = final_state.get("history", self.history)
        
        display(Markdown("---"))
        display(Markdown("### Final Answer"))
        display(Markdown(f'<span style="color: #4CAF50;">{final_answer}</span>'))
        
        return final_answer
    
    def reset_session(self):
        """Start a new session"""
        self.session_id = str(uuid.uuid4())
        self.history = []
        display(Markdown("**Session reset**"))

# Create chatbot instance
chatbot = ReasoningChatbot()

## Example 1: Simple Question

In [ ]:
# Simple question - will show at least 2 reasoning steps
answer = chatbot.ask("What is the capital of France?")

## Example 2: Complex Question

In [ ]:
# Complex question - should trigger more reasoning steps
answer = chatbot.ask("Explain how photosynthesis works and why it's important for life on Earth.")

## Example 3: Follow-up Question

In [ ]:
# Follow-up that uses conversation history
answer = chatbot.ask("What would happen if this process stopped working?")

## Example 4: Math Problem

In [ ]:
# Math problem requiring step-by-step reasoning
answer = chatbot.ask(
    "If a car travels 120 miles in 2 hours, then slows down to half its speed for the next 3 hours, "
    "how far did it travel in total?"
)

## Example 5: Probability Question

### The Dice Game
You play a game where you roll a fair six-sided die. You win the dollar amount shown on the die. However, after seeing your first roll, you have the option to reroll once. What is the maximum amount you should pay to play this game?

**Answer**: $4.25. The optimal strategy is to reroll if you get 1, 2, or 3 (below the expected value of 3.5). Expected value: (1/6)(4+5+6) + (1/2)(3.5) = 4.25

In [ ]:
# Probability/game theory question
answer = chatbot.ask(
    "You play a game where you roll a fair six-sided die and win the dollar amount shown. "
    "After seeing your first roll, you can choose to reroll once. "
    "What is the maximum amount you should pay to play this game?"
)

## Example 6: Reasoning About Code

In [ ]:
# Code analysis question
answer = chatbot.ask(
    "What's the time complexity of bubble sort and why is it generally not used in production systems?"
)

## Reset Session
Start a fresh conversation without history:

In [ ]:
# Reset to start fresh
chatbot.reset_session()

## Example 7: New Session Question

In [ ]:
# This won't have access to previous conversation history
answer = chatbot.ask("What did we discuss earlier?")